In [2]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range

In [4]:
pickle_file = 'sudoku_ml_data'
with open(pickle_file, 'rb') as f:
    save = pickle.load(f)
    locals().update(save)
    del save

In [7]:
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 14) (200000,)
Validation set (10000, 28, 14) (10000,)
Test set (10000, 28, 14) (10000,)


In [8]:
image_size = (28, 14)
num_labels = 9
num_channels = 1

def reformat(dataset, labels):
  dataset = dataset.reshape( (-1, image_size[0], image_size[1], num_channels) ).astype(np.float32)
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels

train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 14, 1) (200000, 9)
Validation set (10000, 28, 14, 1) (10000, 9)
Test set (10000, 28, 14, 1) (10000, 9)


In [9]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

In [22]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64

hyper = {'learn_rate': 0.1
         ,'decay_rate': 0.9, 'decay_steps': 200
        }
graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size[0], image_size[1], num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  global_step = tf.Variable(0)
  if 'decay_steps' in hyper and 'decay_rate' in hyper:
    learning_rate = tf.train.exponential_decay(hyper['learn_rate'], global_step, 
                                               decay_steps=hyper['decay_steps'],
                                               decay_rate=hyper['decay_rate'],
                                               staircase=True)
  else:
    learning_rate = hyper['learn_rate']
 
  layer1_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, num_channels, depth], stddev=0.1))
  layer1_biases = tf.Variable(tf.zeros([depth]))

  layer2_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, depth, depth], stddev=0.1))
  layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))

  layer3_weights = tf.Variable(tf.truncated_normal(
      [image_size[0] // 4 * (image_size[1] // 4 + 1) * depth, num_hidden], stddev=0.1))
  layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))

  layer4_weights = tf.Variable(tf.truncated_normal(
      [num_hidden, num_labels], stddev=0.1))
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
  # Model.
  def model(data, type):
    # data: [batch width height channel] = [n 28 28 1]
    # layer1_weights aka kernel size [fh:5, fw:5, in_ch:1, out_ch:16]
    # strides [batch:1,w:2,h:2,ch:1]
    conv = tf.nn.conv2d(data, layer1_weights, [1, 1, 1, 1], padding='SAME')
    print("Conv 1 shape", conv.shape, '(n, 28, 14, 16)')
    conv = tf.nn.max_pool(conv, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')
    print("C+P  1 shape", conv.shape, '(n, 14, 7, 16)')
    # layer1_biases: [16], default: 0
    hidden = tf.nn.relu(conv + layer1_biases)

    if type == 'train':
        hidden = tf.nn.dropout(hidden, 0.5)
    
    # layer2_weights aka kernel size [fh:5, fw:5, in_ch:16, out_ch:16]
    conv = tf.nn.conv2d(hidden, layer2_weights, [1, 1, 1, 1], padding='SAME')
    conv = tf.nn.max_pool(conv, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')
    print("Conv 2 shape", conv.shape, '(n, 7, 4, 16)')
    hidden = tf.nn.relu(conv + layer2_biases)
    shape = hidden.get_shape().as_list()
    
    reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
    print("reshape     ", reshape.shape, '(n, 448)')
    
    hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)

    if type == 'train':
        hidden = tf.nn.dropout(hidden, 0.5)
    
    print("hidden     ", hidden.shape, '(n, 64)')
    return tf.matmul(hidden, layer4_weights) + layer4_biases
    # num_labels
  
  # Training computation.
  logits = model(tf_train_dataset, 'train')
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
    
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset, 'valid'))
  test_prediction = tf.nn.softmax(model(tf_test_dataset, 'test'))

Conv 1 shape (16, 28, 14, 16) (n, 28, 14, 16)
C+P  1 shape (16, 14, 7, 16) (n, 14, 7, 16)
Conv 2 shape (16, 7, 4, 16) (n, 7, 4, 16)
reshape      (16, 448) (n, 448)
hidden      (16, 64) (n, 64)
Conv 1 shape (10000, 28, 14, 16) (n, 28, 14, 16)
C+P  1 shape (10000, 14, 7, 16) (n, 14, 7, 16)
Conv 2 shape (10000, 7, 4, 16) (n, 7, 4, 16)
reshape      (10000, 448) (n, 448)
hidden      (10000, 64) (n, 64)
Conv 1 shape (10000, 28, 14, 16) (n, 28, 14, 16)
C+P  1 shape (10000, 14, 7, 16) (n, 14, 7, 16)
Conv 2 shape (10000, 7, 4, 16) (n, 7, 4, 16)
reshape      (10000, 448) (n, 448)
hidden      (10000, 64) (n, 64)


In [23]:
num_steps = 16001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print('step %d: Minibatch loss: %f, ' % (step, l), end=" ")
      print('acc: %.1f%%,' % accuracy(predictions, batch_labels), end=" ")
      print('Validation acc: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  #print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
step 0: Minibatch loss: 3.004710,  acc: 0.0%, Validation acc: 11.0%
step 500: Minibatch loss: 2.117218,  acc: 37.5%, Validation acc: 48.1%
step 1000: Minibatch loss: 0.141483,  acc: 93.8%, Validation acc: 94.0%
step 1500: Minibatch loss: 0.006962,  acc: 100.0%, Validation acc: 97.7%
step 2000: Minibatch loss: 0.016487,  acc: 100.0%, Validation acc: 86.6%
step 2500: Minibatch loss: 0.038468,  acc: 100.0%, Validation acc: 99.2%
step 3000: Minibatch loss: 0.226195,  acc: 93.8%, Validation acc: 98.0%
step 3500: Minibatch loss: 1.199474,  acc: 75.0%, Validation acc: 98.3%
step 4000: Minibatch loss: 0.002801,  acc: 100.0%, Validation acc: 96.6%
step 4500: Minibatch loss: 0.004697,  acc: 100.0%, Validation acc: 93.5%
step 5000: Minibatch loss: 0.002156,  acc: 100.0%, Validation acc: 98.9%
step 5500: Minibatch loss: 0.016263,  acc: 100.0%, Validation acc: 99.5%
step 6000: Minibatch loss: 0.032724,  acc: 100.0%, Validation acc: 98.4%
step 6500: Minibatch loss: 0.155455,  acc: 93.8%,